# 案例: 多水库系统的中央调度

本案例演示了`CentralDispatcher`（中央调度器）如何协调一个由两个串联水库组成的复杂系统。调度器会监控两个水库的水位，并根据一套全局规则，动态调整每个水库本地PID控制器的目标水位，以应对上游来水变化，实现系统层面的削峰和蓄水。

### 1. 导入库并加载配置

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.central_coordination.dispatch.central_dispatcher import CentralDispatcher
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController

CONFIG_PATH = 'examples/multi_reservoir_dispatch.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义并运行仿真

In [ ]:
def run_multi_reservoir_simulation(cfg):
    # 1. 初始化
    bus = MessageBus()
    dt = cfg['simulation']['dt']
    topics = cfg['messaging_topics']
    
    # 2. 创建中央调度器
    dispatcher = CentralDispatcher(
        agent_id="system_dispatcher", message_bus=bus,
        state_subscriptions={"res1": topics['res1_state'], "res2": topics['res2_state']},
        command_topics={"res1_control": topics['res1_command'], "res2_control": topics['res2_command']},
        rules=cfg['dispatcher_rules']
    )

    # 3. 创建水库1及其组件
    res1_cfg = cfg['reservoir1']
    res1 = Reservoir("res1", res1_cfg['initial_state'], res1_cfg['params'])
    gate1 = Gate("gate1", {'opening': 0.5}, res1_cfg['gate_params'])
    pid1_params = res1_cfg['pid_params']
    pid1 = PIDController(setpoint=cfg['dispatcher_rules']['res1_normal_setpoint'], min_output=0, **pid1_params)

    # 4. 创建水库2及其组件
    res2_cfg = cfg['reservoir2']
    res2 = Reservoir("res2", res2_cfg['initial_state'], res2_cfg['params'])
    gate2 = Gate("gate2", {'opening': 0.5}, res2_cfg['gate_params'])
    pid2_params = res2_cfg['pid_params']
    pid2 = PIDController(setpoint=cfg['dispatcher_rules']['res2_normal_setpoint'], min_output=0, **pid2_params)

    # 5. 订阅调度器的指令
    bus.subscribe(topics['res1_command'], lambda msg: pid1.set_setpoint(msg['new_setpoint']))
    bus.subscribe(topics['res2_command'], lambda msg: pid2.set_setpoint(msg['new_setpoint']))

    # 6. 仿真循环
    print("\n--- 开始仿真 ---")
    history = []
    inflows1 = cfg['simulation']['inflows_res1']

    for t, inflow1 in enumerate(inflows1):
        # 发布状态给调度器
        bus.publish(topics['res1_state'], res1.get_state())
        bus.publish(topics['res2_state'], res2.get_state())

        # 调度器决策
        dispatcher.run(current_time=t*dt)

        # 本地控制器计算并执行
        action1 = pid1.compute_control_action({'process_variable': res1.get_state()['water_level']}, dt)
        gate1.step({'opening': action1}, dt)
        
        # 系统物理状态更新
        inflow2 = gate1.get_state()['outflow']
        res1.set_inflow(inflow1)
        res1.step({'outflow': inflow2}, dt)
        
        action2 = pid2.compute_control_action({'process_variable': res2.get_state()['water_level']}, dt)
        gate2.step({'opening': action2}, dt)
        
        inflow3 = gate2.get_state()['outflow']
        res2.set_inflow(inflow2)
        res2.step({'outflow': inflow3}, dt)

        # 记录历史
        history.append({
            'time': t, 'res1_level': res1.get_state()['water_level'], 'res2_level': res2.get_state()['water_level'],
            'pid1_setpoint': pid1.setpoint, 'pid2_setpoint': pid2.setpoint
        })
    
    results_df = pd.DataFrame(history)
    print("\n--- 仿真结果 ---")
    print(results_df)
    return results_df

results = run_multi_reservoir_simulation(config)

### 3. 可视化结果

下图展示了两个水库的水位以及它们的PID控制器目标水位（setpoint）随时间的变化。可以清晰地看到，当上游水库（res1）面临巨大来水时，中央调度器是如何通过调整两个水库的目标水位来协同应对的。

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15, 12), sharex=True)
fig.suptitle('多水库中央调度结果', fontsize=16)

# 水库1的图
ax[0].plot(results['time'], results['res1_level'], 'b-', label='水库1 水位 (m)')
ax[0].plot(results['time'], results['pid1_setpoint'], 'b--', label='水库1 目标水位 (m)')
ax[0].set_ylabel('水位 (m)')
ax[0].legend()
ax[0].grid(True)
ax[0].set_title('上游水库1')

# 水库2的图
ax[1].plot(results['time'], results['res2_level'], 'g-', label='水库2 水位 (m)')
ax[1].plot(results['time'], results['pid2_setpoint'], 'g--', label='水库2 目标水位 (m)')
ax[1].set_xlabel('时间步')
ax[1].set_ylabel('水位 (m)')
ax[1].legend()
ax[1].grid(True)
ax[1].set_title('下游水库2')

plt.show()